# Saúde (Health)

In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import numpy as np
from utils.manual_data import manual_data_path

In [2]:
onibus_diesel_dia31= pd.read_csv(manual_data_path('linestring_2025-12-31_silver.csv'))
df_onibus=onibus_diesel_dia31
df_onibus['geometry'] = df_onibus['geometry'].apply(wkt.loads)
gdf_onibus =gpd.GeoDataFrame(
    df_onibus, 
    geometry= 'geometry',
    crs="EPSG:31983"
)


np.random.seed(20260129)
sample_onibus=gdf_onibus.sample(100).copy()

## Definir valores dos buffers (Defining Buffer values)
### PT-BR
O valor dos buffers para NOx e MP serão definidos futuramente, após conversarmos com especialistas da área da saúde.
### ENG
The buffer values for NOx and MP will be defined in the future, after a talk with people specialized in the area

In [3]:
BUFFER_NOX=5
BUFFER_MP=10

In [4]:
#gdf_buffer_nox['geometry'] = gdf_buffer.geometry.buffer(BUFFER_NOX)
#gdf_buffer_mp['geometry'] = gdf_buffer.geometry.buffer(BUFFER_MP)

## Overlay Setores Censitários

### PT-BR
Dados dos setores censitários com atributos dos resultados de população e domicílios (Censo de 2022) do estado de São Paulo, extraídos do IBGE.
### ENG
Census sectors data, with population and households attributes (2022 Census) for São Paulo state, extracted from IBGE

In [5]:
from setores_censitarios import setores_censitarios_final
setores_censitarios_sp = setores_censitarios_final()

Retornando arquivo salvo


In [6]:
def overlay_setores_censitarios(
    gdf_buffer, 
    BUFFER,
#    setores_censitarios_sp:gpd.GeoDataFrame()=setores_censitarios_sp
):
    #overlay
    gdf_overlay = gpd.overlay(
        setores_censitarios_sp,
        gdf_buffer,
        how = 'intersection'
    )
    gdf_overlay = gdf_overlay.explode(index_parts = False).reset_index(drop = True)

    #tirar geometrias pequenininhas
    print(f'Lenght before drop: {len(gdf_overlay)}')
    
    debuff_value= -(BUFFER**(1/2))
    gdf_overlay['debuffed'] = gdf_overlay.buffer(debuff_value)
    gdf_overlay['debuffed_area']= gdf_overlay['debuffed'].area
    gdf_overlay = gdf_overlay[gdf_overlay['debuffed_area']>0]
    gdf_overlay = gdf_overlay.drop(columns = {'debuffed', 'debuffed_area'})
    
    print(f'Lenght after drop: {len(gdf_overlay)}')

    return gdf_overlay

## Cálculo da População Afetada

In [7]:
def calculo_pop_afetada():

    setores_censitarios_sp['area_setor'] = setores_censitarios_sp.geometry.area
    setores_area = setores_censitarios_sp[['CD_SETOR', 'area_setor', 'pop']]

    gdf_overlay['area_intersecao'] = gdf_overlay.geometry.area
    gdf_overlay = gdf_overlay.merge(setores_area,
                                on = 'CD_SETOR',
                                how = 'left')

    gdf_overlay = gdf_overlay.drop(columns = ['pop_y'])
    gdf_overlay = gdf_overlay.rename(columns = {'pop_x': 'pop'})
    gdf_overlay['pop_afetada'] = round(
        (gdf_overlay['pop'] * (
            gdf_overlay['area_intersecao'] / gdf_overlay['area_setor']
        ))
    ).astype(int)
    gdf_overlay = gdf_overlay.rename(columns = {'area_intersecao': 'area_inter', 
                                            'pop_afetada': 'pop_buffer'})

In [8]:
def get_pop_por_setor():
#alterar pra ficar pro setor todo, o valor dos dois buffers...
#fazer o mesmo pros ônibus
    setores_uniques = (
        gdf_overlay[['CD_SETOR', 'pop']]
        .drop_duplicates(subset='CD_SETOR')
        .rename(columns={'CD_SETOR': 'name'})
        .reset_index(drop=True)
    )
    for i, s in setores_uniques.iterrows():
        cd = s['name']
        setor_x = gdf_overlay.loc[gdf_overlay['CD_SETOR'] == cd]
        #soma pop afetada
        setores_uniques.loc[setores_uniques['name'] == cd, 'soma_pop_afetada'] = sum(
            setor_x['pop_buffer']
        )
    
    setores_uniques['subtrac_pop'] = setores_uniques['pop'] - setores_uniques['soma_pop_afetada']
    return setores_uniques
            

def get_pop_por_onibus():
    gdf_buffer['pop_afetada'] = [
        gdf_overlay.loc[gdf_overlay['codigo_onibus'] == cb, 'pop_buffer'].sum()
        for cb in gdf_buffer['codigo_onibus']
    ]
    gdf_buffer['pop_afetada'] = gdf_buffer['codigo_onibus'].map(
        gdf_overlay.groupby('codigo_onibus')['pop_buffer'].sum()
    )
    gdf_buffer